In [47]:
import matplotlib.pyplot as plt
import pandas as pd

import numpy as np
import datetime as dt

import wbdata
# we hates modules that don't apppear on anaconda!
# pip install wbdata
import datetime
import csv
import statsmodels.api as sm
from statsmodels.api import add_constant

# from pandas.stats.plm import PanelOLS
# deprecated

from linearmodels import PanelOLS
# pip install linearmodels

In [43]:
import os
BES_data_folder   = "../BES_analysis_data/"
if not os.path.exists( BES_data_folder ):
    os.makedirs( BES_data_folder )
    
Legatum_folder = BES_data_folder + os.sep + "Legatum" + os.sep
encoding = "ISO-8859-1"

In [58]:
## pull in and clean fdi stock data ##

# fdi = pd.read_excel("us_fdiflowsstock_29777289490548.xls", header = 4)
# save as csv! xls is a pain
# header=4 -> header=5
fdi_raw = pd.read_csv(Legatum_folder+"us_fdiflowsstock_29777289490548.csv", header=5, encoding=encoding)
fdi = fdi_raw.iloc[1:]
fdi = fdi_raw.transpose()

fdi.columns = fdi.iloc[0]
fdi = fdi.iloc[1:]
fdi = fdi.stack()
fdi = fdi.reset_index()

fdi.columns = ['Year', 'country', 'FDI stock']
fdi['Year'] = fdi['Year'].astype('int', errors='raise')
fdi['date'] = pd.to_datetime(fdi['Year'], errors='ignore', format = "%Y")
fdi['FDI stock'] = pd.to_numeric(fdi['FDI stock'], errors='coerce')
fdi['country'] = fdi['country'].str.strip()
##### sort -> sort_values
fdi = fdi.sort_values(["country", "date"])

## pull year from annual data to ease matching to rest ## 
fdi['year'] = pd.DatetimeIndex(fdi['date']).year
fdi = fdi.set_index(["country", "year"])
del fdi['Year']
del fdi['date']

## pull remaining data from the world bank ##

indicators = {"BN.KLT.DINV.CD" : "FDI", "NY.GDP.MKTP.CD" : "GDP per capita", "SH.XPD.CHEX.PC.CD": "Health expenditure", "FS.AST.PRVT.GD.ZS" : "Domestic credit", "TX.VAL.FUEL.ZS.UN" : "Fuel exports", "TX.VAL.MMTL.ZS.UN" : "Ore and metal", "se.prm.cmpt.zs": "Primary school completion"}
df = wbdata.get_dataframe(indicators, country="all", data_date=(datetime.datetime(2010, 1, 1), datetime.datetime(2018, 1, 1)),  convert_date=True)

## pull year from annual data to ease matching to fdi ## 
df = df.reset_index()
df['year'] = pd.DatetimeIndex(df['date']).year
df = df.set_index(["country", "year"])
del df['date']

df = df.join(fdi)
df = df.dropna()

df['GDP per capita log'] = df['GDP per capita'].apply(np.log)
df['FDI log'] = df['FDI stock'].apply(np.log)

## this caps the number of obs per country at 4 - gives you 379 rows (their n = 384)##

df = df.reset_index()
# df_2 = df.groupby("country").head(4)
# df = df_2
df = df.set_index(["country", "year"])

In [59]:
df
# passes eyeball comparison with xtophercooks notebook

Domestic credit           FDI  Fuel exports  \
country             year                                                
Albania             2015        37.151375 -9.100003e+08      8.806638   
                    2014        39.261063 -1.073931e+09      1.567523   
                    2013        39.787091 -1.225802e+09     31.003052   
                    2012        40.803822 -8.350794e+08     26.590781   
                    2011        41.153526 -8.461392e+08     21.191947   
                    2010        39.083743 -1.043720e+09     17.973227   
Algeria             2015        21.725470  4.989344e+08     94.325860   
                    2014        18.351978 -1.521892e+09     95.764232   
                    2013        16.497891 -1.963635e+09     96.722000   
                    2012        14.025307 -1.541845e+09     97.135491   
                    2011        13.715511 -2.037319e+09     97.196901   
                    2010        15.208241 -2.081136e+09     97.345998   
Angola              2011        20.179427  5.116413e+09     96.556768   
                    2010        20.202350  4.567641e+09     96.534547   
Antigua and Barbuda 2015        51.219357 -9.564097e+07      0.000000   
                    2014        60.348025 -4.001160e+07      0.000000   
Argentina           2015        14.414233 -1.088376e+10      2.611328   
                    2014        13.823769 -3.144794e+09      4.751054   
                    2013        15.729092 -8.931689e+09      4.758954   
                    2012        15.212815 -1.426908e+10      6.374852   
                    2011        14.008715 -9.351931e+09      6.119545   
                    2010        12.690106 -1.036796e+10      8.128336   
Armenia             2015        45.630783 -1.615249e+08      6.419701   
Austria             2015        85.835374  6.361436e+09      1.967985   
                    2014        87.577525 -2.586685e+09      2.386196   
                    2013        92.557752  1.057847e+10      2.658385   
                    2012        94.223799  1.323003e+10      3.624504   
                    2011        96.061099  1.514534e+10      3.371345   
Azerbaijan          2015        38.520786 -8.382520e+08     87.023136   
                    2014        30.638025 -2.443393e+09     92.638413   
...                                   ...           ...           ...   
Tonga               2012        30.513391 -2.869190e+05      0.000628   
Trinidad and Tobago 2010        30.845461 -5.494000e+08     66.122752   
Tunisia             2015        79.338136 -9.705219e+08      7.211910   
                    2014        78.116264 -1.024754e+09     13.189847   
                    2013        76.956587 -1.058623e+09     15.216031   
                    2010        69.111924 -1.334498e+09     14.177306   
Turkey              2015        66.827360 -1.290600e+10      3.083329   
                    2014        63.791062 -6.069000e+09      3.761938   
                    2012        52.228001 -9.638000e+09      5.285242   
                    2011        49.424062 -1.381200e+10      4.699200   
                    2010        44.654785 -7.617000e+09      3.904515   
Uganda              2014        14.872813 -1.031546e+09      2.114100   
                    2013        13.913392 -1.143207e+09      1.073146   
                    2012        13.983883 -1.159051e+09      6.673171   
                    2011        15.803888 -9.061404e+08      5.773000   
                    2010        13.341297 -5.066584e+08      1.209993   
Ukraine             2014        75.228846 -2.990000e+08      3.389940   
                    2013        73.463387 -4.079000e+09      4.200171   
                    2012        69.579246 -7.195000e+09      5.049094   
                    2011        71.075042 -7.015000e+09      8.112370   
                    2010        78.646186 -5.759000e+09      6.925322   
United Kingdom      2015       132.787995 -1.145039e+11      7.654935   
Uruguay    

In [83]:
Y = df["GDP per capita log"]
X = df[["FDI log", "Health expenditure", "Domestic credit", "Fuel exports", "Ore and metal", "Primary school completion"]]
X = add_constant(X)

# regression=PanelOLS(y=Y, x=X, time_effects=True)
# regression

# linearmodel PanelOLS is *not completely identical* to pandas
# their explanation of the differences
# https://bashtage.github.io/linearmodels/doc/panel/pandas.html

PanelOLS(Y, X, time_effects=True).fit()

Dep. Variable:,GDP per capita log,R-squared:,0.7922
Estimator:,PanelOLS,R-squared (Between):,0.8240
No. Observations:,494,R-squared (Within):,-2.7035
Date:,"Tue, Sep 25 2018",R-squared (Overall):,0.7924
Time:,12:20:26,Log-likelihood,-658.72
Cov. Estimator:,Unadjusted,,
,,F-statistic:,306.34
Entities:,112,P-value,0.0000
Avg Obs:,4.4107,Distribution:,"F(6,482)"
Min Obs:,1.0000,,
Max Obs:,6.0000,F-statistic (robust):,306.34


In [55]:
PanelOLS(Y, X, time_effects=False).fit()

Dep. Variable:,GDP per capita log,R-squared:,0.7983
Estimator:,PanelOLS,R-squared (Between):,0.8248
No. Observations:,379,R-squared (Within):,-3.0355
Date:,"Tue, Sep 25 2018",R-squared (Overall):,0.7983
Time:,10:56:45,Log-likelihood,-505.36
Cov. Estimator:,Unadjusted,,
,,F-statistic:,245.46
Entities:,112,P-value,0.0000
Avg Obs:,3.3839,Distribution:,"F(6,372)"
Min Obs:,1.0000,,
Max Obs:,4.0000,F-statistic (robust):,245.46


In [71]:
# df2 = df.reset_index().set_index(['year','country'])


Y = df["GDP per capita log"]
X = df[["FDI log", "Health expenditure", "Domestic credit", "Fuel exports", "Ore and metal", "Primary school completion"]]
X = add_constant(X)

# regression=PanelOLS(y=Y, x=X, time_ef
PanelOLS(Y, X, time_effects=True, entity_effects=True).fit()

Dep. Variable:,GDP per capita log,R-squared:,0.3139
Estimator:,PanelOLS,R-squared (Between):,0.3155
No. Observations:,494,R-squared (Within):,0.4210
Date:,"Tue, Sep 25 2018",R-squared (Overall):,0.3246
Time:,11:31:22,Log-likelihood,678.43
Cov. Estimator:,Unadjusted,,
,,F-statistic:,28.292
Entities:,112,P-value,0.0000
Avg Obs:,4.4107,Distribution:,"F(6,371)"
Min Obs:,1.0000,,
Max Obs:,6.0000,F-statistic (robust):,28.292


,country,year,Domestic credit,FDI,Fuel exports,GDP per capita,Health expenditure,Ore and metal,Primary school completion,FDI stock,GDP per capita log,FDI log
0,Albania,2015,37.151375,-9.100003e+08,8.806638,1.138693e+10,265.903236,7.545843,104.468193,4.336516e+03,23.155732,8.374827
1,Albania,2014,39.261063,-1.073931e+09,1.567523,1.322824e+10,314.862629,4.582160,105.002121,4.295366e+03,23.305620,8.365292
2,Albania,2013,39.787091,-1.225802e+09,31.003052,1.277628e+10,277.668927,9.783993,90.974411,3.930695e+03,23.270856,8.276571
3,Albania,2012,40.803822,-8.350794e+08,26.590781,1.231978e+10,247.266691,10.912702,98.911697,4.303264e+03,23.234472,8.367129
4,Albania,2011,41.153526,-8.461392e+08,21.191947,1.289087e+10,248.172587,11.903426,93.277008,4.399132e+03,23.279785,8.389163
5,Albania,2010,39.083743,-1.043720e+09,17.973227,1.192695e+10,204.213137,12.829349,89.237343,3.254943e+03,23.202067,8.087930
6,Algeria,2015,21.725470,4.989344e+08,94.325860,1.658743e+11,291.635898,0.309081,105.436417,2.623229e+04,25.834496,10.174746
7,Algeria,2014,18.351978,-1.521892e+09,95.764232,2.138100e+11,357.898943,0.179691,108.280853,2.681960e+04,26.088354,10.196888
8,Algeria,2013,16.497891,-1.963635e+09,96.722000,2.097550e+11,330.223372,0.167012,105.913170,2.531287e+04,26.069206,10.139068
9,Algeria,2012,14.025307,-1.541845e+09,97.135491,2.090590e+11,333.936505,0.246081,101.450180,2.361998e+04,26.065882,10.069848


In [ ]:
############## this model makes no sense to me
### try a dump tree based regression

In [ ]:
df_flat = df.reset_index()
Y = df_flat["GDP per capita log"]
X = df_flat[["FDI log", "Health expenditure", "Domestic credit", "Fuel exports", "Ore and metal", "Primary school completion"]]
X = add_constant(X)

In [84]:
import shap
import xgboost as xgb
from sklearn.preprocessing import Imputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
from sklearn.linear_model import ElasticNet
from xgboost import XGBClassifier, XGBRegressor
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error, explained_variance_score, r2_score

In [85]:
alg = XGBRegressor(
 learning_rate =0.04,
 n_estimators= 20,
 max_depth=3,
 min_child_weight=1,
 gamma=0,
 subsample=0.75,
 colsample_bytree=0.9,
 colsample_bylevel=.85,
 objective= 'reg:linear',
 scale_pos_weight=1.0,
 reg_alpha=0,
 reg_lambda=1,
 njobs=3,
 seed=27)

In [ ]:
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size,
                                                    random_state=seed, stratify=round(target))



eval_set = [(X_test, y_test)]

sample_weight = X_train[weight_var].values
sample_weight_eval_set = X_test[weight_var].values
X_train.drop(weight_var, axis=1, inplace=True)
X_test.drop(weight_var, axis=1, inplace=True)

alg.fit(X_train, y_train, eval_metric='rmse', 
        early_stopping_rounds=alg.get_params()['n_estimators']*.1, eval_set=eval_set,
        verbose=True, sample_weight= sample_weight)